In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

import requests
import csv
import os

import numpy as np
import pandas as pd
from numpy import random
from time import sleep

import parser

In [2]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()
driver.get('https://bringatrailer.com/toyota/land-cruiser-fj60-fj62/')

In [3]:
# extend page
while True:
    try:
        button = driver.find_element(By.XPATH , "//div[@class='auctions-footer auctions-footer-previous']/*")
        button.click()
    except NoSuchElementException:
        break
    sleeptime = random.uniform(3, 5)
    sleep(sleeptime)

In [4]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [5]:
listings = soup.find('div', {'class': 'blocks'}).find_all('div', {'class': 'block'})
links = []
for listing in listings:
    link = listing.a.get('href')
    links.append(link)
# remove below link because it was REMOVED from auction, but page still exists (i.e., auction did not finish)
links.remove('https://bringatrailer.com/listing/1984-toyota-land-cruiser-fj60-7/')

In [6]:
parser = parser.BATParser()
with open ('land_cruiser_scrape.csv', 'w', encoding='utf-8') as csv_file:
    # format csv
    csv_writer = csv.writer(csv_file)
    # header
    csv_writer.writerow(['title','price','sell_date', 'no_views','no_watchers','no_comments', 'no_bids', 'seller_state', 
                         'year', 'series', 'miles','engine','trans','exterior_paint','interior', 'misc', 'url'])
    for url in links:
        source = requests.get(url).text
        
        scraped_page = parser.parse(source)
        scraped_page.append(url)
        csv_writer.writerow(scraped_page)

        sleeptime = random.uniform(1, 10)
        sleep(sleeptime)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
# https://bringatrailer.com/listing/1985-toyota-land-cruiser-fj60-2/ 

In [ ]:
# https://bringatrailer.com/listing/1984-toyota-land-cruiser-fj60-7/
# causes machine to break